In [1]:
import os
import shutil
from pathlib import Path

import numpy as np
import pandas as pd

import json
from PIL import Image
from tqdm import tqdm


from DataExtractor.downloader import download_dataset
from DataExtractor.dataset_to_dataframe import ConvertICDARDatasetToDataframe
from DataAugmentation.augmentation import Augmentation
from DataExtractor.yolo_converter import ICDARYOLOConverter
from DataSplitter.kfold import DataFrameKFoldSplitter

# Download do Dataset

In [2]:
# realiza o download do dataset

download_dataset()

Baixando o dataset... espere!
Baixado 6125166592 bytes de indefinido...
Extraindo dataset...
Download do dataset!


# Extração de Metadados em DataFrame

In [3]:
# gera um dataframe contendo metadados sobre o dataset como o caminhos até as imagens, ss dimensões das imagens, 
# as anotações dos pontos (x,y) dos vértices dos polígonos das máscaradas extraídas a partir dos arquivos XML, etc.

converter = ConvertICDARDatasetToDataframe(
    images_path="./dataset/training/TRACKB1/ground_truth",
    labels_path="./dataset/training/TRACKB1/ground_truth"
)

df = converter.generate_dataframe()
df.to_csv('dataset.csv',  index=False)

Associando pares... : 100%|██████████| 600/600 [00:00<00:00, 727.72it/s]


Gerando DataFrame do conjnuto de dados...


Extraindo as anotações...: 100%|██████████| 600/600 [00:13<00:00, 43.95it/s]


# Redimensionamento das Imagens

In [4]:
# realiza a leitura dos metadados do conjunto de dados no arquivo dataset.csv
dataset_df = pd.read_csv('dataset.csv')

# cria uma nova pasta para armazenar as imagens redimensionadas 
os.makedirs('resized_dataset', exist_ok=True)


In [5]:
# Redimensiona as imagens para o tamanho 640x640, além de redimensionar as anotações respectivamente

new_width, new_height = 640, 640
total_rows = dataset_df.shape[0]

new_masks = []
new_paths = []

print(f'Redimensionando as imagens e anotaçãoes para {new_width}X{new_height}')

pbar = tqdm(dataset_df.iterrows(),total=dataset_df.shape[0])

for _, dataset_row in pbar: 

    
    image_path  = dataset_row['image_path']
    masks = json.loads(dataset_row['xy'])

    image_filename = os.path.basename(image_path)
    output_path = f'./resized_dataset/{image_filename}'
    
    image = Image.open(image_path)
    image = np.asarray(image)

    pbar.set_description(f'Convertendo imagem {image_filename} ({image.shape[1]}X{image.shape[0]}) -> ({new_width}X{new_height})')
    
    resized_image, resized_masks = \
        Augmentation.resize_image(image, new_width, new_height, masks=masks)
    
    new_masks.append(resized_masks)
    new_paths.append(output_path)

    Augmentation.save_image(resized_image, output_path)


Redimensionando as imagens e anotaçãoes para 640X640


Convertendo imagem cTDaR_t00930.jpg (2480X3830) -> (640X640): 100%|██████████| 600/600 [01:21<00:00,  7.40it/s] 


In [6]:
# remove o diretório do dataset original, já que ele não será necessário
shutil.rmtree('dataset/')

In [7]:
# descarta o arquivo dataset.csv, já que ele não é necessário também
os.remove('dataset.csv')

In [8]:
# armazena os metadados do conjunto redimensionado em um arquivo csv
resized_dataset_df = pd.DataFrame(
    {'image_path':new_paths,
    'xy':new_masks}
)

resized_dataset_df['image_width'] = new_width
resized_dataset_df['image_height'] = new_height

resized_dataset_df.to_csv('resized_dataset.csv', index=False)

# Geração das Anotações YOLO

In [9]:
# leitura do metadados do conjunto de dados redimensionado
resized_dataset_df = pd.read_csv('resized_dataset.csv')

In [10]:
# geração dos rótulos YOLO em arquivos txt no mesmo diretório das respectivas imagens

new_masks = []
txt_paths = []

for _, resized_dataset_row in tqdm(resized_dataset_df.iterrows(),  
                                   total = resized_dataset_df.shape[0],
                                   desc='Gerando anotações YOLO em txt...') :
    
    image_width = resized_dataset_row['image_width']
    image_height = resized_dataset_row['image_height']
    image_path = resized_dataset_row['image_path']
    masks = resized_dataset_row['xy']
    masks = json.loads(masks)

    image_filename = os.path.splitext(os.path.basename(image_path))[0]
    image_path = Path(os.path.dirname(image_path))
    txt_path = image_path/f'{image_filename}.txt' 

    
    yolo_normalized_masks = ICDARYOLOConverter.process_masks(masks, image_width, image_height, txt_path)

    new_masks.append(yolo_normalized_masks)
    txt_paths.append(txt_path.as_posix())



Gerando anotações YOLO em txt...: 100%|██████████| 600/600 [00:01<00:00, 316.29it/s]


In [11]:
# adiciona as máscarar convertidas para o formato YOLO e os respectivos caminhos para os arquivos TXT com as anotações salva
resized_dataset_df['yolo_xy'] = new_masks
resized_dataset_df['yolo_txt_path'] = txt_paths

In [12]:
# atualiza os metadados do conjunto redimensionado
resized_dataset_df.to_csv('resized_dataset.csv')

# Geração dos Folds para Validação Cruzada

In [13]:
# lê novamente os metadados do conjunto redimensionado em CSV
resized_dataset_df = pd.read_csv('resized_dataset.csv')

In [14]:
# realiza a divisão do dataframe em folds de treinamento e validação
kfolder = DataFrameKFoldSplitter(resized_dataset_df, n_splits=5, random_state=42)
folds = kfolder.split_folds()

Gerando folds...: 100%|██████████| 5/5 [00:00<00:00, 294.78it/s]


In [15]:
# define o caminho e criação do diretório para o conjunto com os folds
dataset_folds_path = Path('dataset_folds/')
os.makedirs(dataset_folds_path, exist_ok=True)

In [16]:
# realiza a criação dos diretórios dos folds e respectiva cópia dos dados na estrutura esperada pela Ultralytics
# cada fold também conta com o arquivo YAML com os caminhos até os dados de treinamento e validação esperado pela Ultralytics 
# por fim, cada fold também conta com um arquvio dataset.csv com os respectivos metadados daquele fold

for index, fold in enumerate(tqdm(folds, total=len(folds), desc='Gerando folds...')):
    images_path = []
    labels_path = []

    dataset_fold_columns = ['image_path', 'label_path', 'split', 'fold', 'image_width', 'image_height', 'xy', 'yolo_xy']
    dataset_fold_df = pd.DataFrame(columns=dataset_fold_columns)

    fold_path = dataset_folds_path/f'fold_{index+1}'
    
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
        
        ICDARYOLOConverter.create_folders(fold_path)

        splits = ['train', 'val']

        for split in splits:
        
            fold_images_path = fold_path/split/'images'
            fold_labels_path = fold_path/split/'labels'

            original_images_path_list = fold[split]['image_path']
            for original_image_path in original_images_path_list:
                shutil.copy(original_image_path , fold_images_path.as_posix())
                
            original_labels_path_list = fold[split]['yolo_txt_path']
            for original_label_path in original_labels_path_list:
                shutil.copy(original_label_path, fold_labels_path.as_posix())


            images_path = [fold_images_path/image_basename for image_basename in os.listdir(fold_images_path)]
            labels_path = [fold_labels_path/label_basename for label_basename in os.listdir(fold_labels_path)]

            temp_df = pd.DataFrame({
                'image_path': images_path, 
                'fold_path': labels_path,
                'split':  fold[split].shape[0] * [split] ,
                'fold':  fold[split].shape[0] * [index+1],
                'image_width': fold[split]['image_width'],
                'image_height': fold[split]['image_height'],
                'xy': fold[split]['xy'],
                'yolo_xy': fold[split]['yolo_xy']})
            
            dataset_fold_df = pd.concat([dataset_fold_df, temp_df], axis=1)


        ICDARYOLOConverter.create_yaml(
            output_dir=fold_path,
            train_fold_path=fold_path/'train',
            val_fold_path=fold_path/'val'
        )

        dataset_fold_df.to_csv(fold_path/'dataset.csv', index=False)




Gerando folds...:   0%|          | 0/5 [00:00<?, ?it/s]


YAML criado em: dataset_folds\fold_1\dataset.yaml


Gerando folds...:  20%|██        | 1/5 [00:27<01:51, 27.87s/it]


YAML criado em: dataset_folds\fold_2\dataset.yaml


Gerando folds...:  40%|████      | 2/5 [00:28<00:35, 11.99s/it]

...

Gerando folds...:  60%|██████    | 3/5 [00:29<00:13,  6.99s/it]


YAML criado em: dataset_folds\fold_4\dataset.yaml


Gerando folds...:  80%|████████  | 4/5 [00:30<00:04,  4.68s/it]


YAML criado em: dataset_folds\fold_5\dataset.yaml


Gerando folds...: 100%|██████████| 5/5 [00:31<00:00,  6.38s/it]


In [17]:
# remove o diretório do dataset redimensionado, já que ele não será necessário
shutil.rmtree('resized_dataset/')

In [18]:
# descarta o arquivo resized_dataset.csv, já que ele não é necessário também
os.remove('resized_dataset.csv')